In [15]:
import numpy as np
import sys
import pickle
from os import mkdir
from os.path import join, isdir
from numpy import linalg as LA
from math import sqrt, inf
from decimal import Decimal
import time
import gym
import envs
from gym import spaces, logger
from scenario_objects import Point, Cell, User, Environment
import plotting
from my_utils import *
import agent
from Astar import *
'''import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
'''# _________ Main training parameters:_________

SHOW_EVERY = 30
LEARNING_RATE = 1.0
DISCOUNT = 0.95
EPSILON = 1.0
EPSILON_DECREMENT = 0.998
EPSILON_MIN = 0.01
EPSILON_MIN2 = 0.4

max_value_for_Rmax = 100
ITERATIONS_PER_EPISODE = 40

# _____________________________________________


# __________________ Main loadings: __________________



In [16]:
env = gym.make('UAVEnv-v0')
MAX_UAV_HEIGHT = env.max_uav_height
n_actions = env.nb_actions
actions_indeces = range(n_actions)
cs_cells = env.cs_cells
cs_cells_coords_for_UAVs = [(cell._x_coord, cell._y_coord) for cell in cs_cells] if DIMENSION_2D==True else [(cell._x_coord, cell._y_coord, cell._z_coord) for cell in cs_cells]
#cells_matrix = env.cells_matrix
action_set_min = env.action_set_min
if (UNLIMITED_BATTERY==False):
    q_table_action_set = env.q_table_action_set
    charging_set = env.charging_set
    come_home_set = env.come_home_set

reset_uavs = env.reset
centroids = env.cluster_centroids
# Scale centroids according to the selected resolution:
env_centroids = [(centroid[0]/CELL_RESOLUTION_PER_COL, centroid[1]/CELL_RESOLUTION_PER_ROW) for centroid in centroids]
print(action_set_min)

[5, 6, 7, 8, 4]


In [17]:
agent=env.agents
agent


In [18]:
states = env.observation_space.shape
actions = env.action_space.n


In [19]:
n_actions

7

In [20]:
actions

7

In [21]:
states

(10, 10)

In [22]:
length, width= env.observation_space.shape

In [23]:
length

10

In [24]:
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

def build_model(height, width, actions):
    model = tensorflow.keras.Sequential()    
    model.add(Dense(240, activation='relu', input_shape=(2,height, width)))
    model.add(Dense(240, activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [25]:
model = build_model(length, width, actions)

In [26]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 2, 10, 240)        2640      
                                                                 
 dense_6 (Dense)             (None, 2, 10, 240)        57840     
                                                                 
 flatten_1 (Flatten)         (None, 4800)              0         
                                                                 
 dense_7 (Dense)             (None, 512)               2458112   
                                                                 
 dense_8 (Dense)             (None, 256)               131328    
                                                                 
 dense_9 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 2,651,719
Trainable params: 2,651,719
No

In [31]:
env.reset().shape

(10, 10)

In [27]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=7)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [29]:
dqn = build_agent(model, actions)
dqn.compile(Adam())
dqn.fit(env, nb_steps=100, visualize=False, verbose=2)

Training for 100 steps ...


ValueError: Error when checking input: expected dense_5_input to have 4 dimensions, but got array with shape (1, 7)

In [ ]:
print(model.output_shape)

In [ ]:
print(actions)

In [ ]:
env.reset =env.reset_uavs